## PRE


In [1]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm
from functools import reduce
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
from itertools import product
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import *

In [2]:
import custom_functions
import functions

In [3]:
# Download the stopwords dataset if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Make a copy of the original DataFrame
processed_df = df.copy()

# stemmer
stemmer = PorterStemmer()

processed_df['description'] = processed_df.description.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.split(' ')]))

In [6]:
stop_words = set(stopwords.words('english'))

# Function to remove stopwords and punctuation from a text
def clean(text):
    """
    The following function returns the filtered element for each column of a dataframe.
    Filtering operation consists in removing punctuation and removing stopwords given text with lower case
    """
    words = word_tokenize(str(text))
    # Remove punctuation using NLTK and string.punctuation
    filtered_words = [word for word in words if word not in string.punctuation + "'’...?'+,-‘“”„•…›€✓"]
    # Remove stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Selecting columns
text_columns = processed_df.select_dtypes(include='object').columns
# Lower
processed_df[text_columns] = processed_df[text_columns].apply(lambda x: x.str.lower() if x.dtype == 'O' else x)
# Applying clean function to all the valid columns
processed_df[text_columns] = processed_df[text_columns].applymap(clean)


C:\Users\Matt\AppData\Local\Temp\ipykernel_17372\1351881474.py:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  processed_df[text_columns] = processed_df[text_columns].applymap(clean)


In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### FEES

In [8]:
raw_fees= pd.DataFrame(processed_df['fees'])

In [9]:
# Missing values for fees
print(sum(raw_fees['fees'].isna()==True))

0


In [10]:
# Your ExchangeRate-API key
api_key = '40f223580924eaf7a1eb4ee0'

# Fetch exchange rates from the API for all currencies against USD
api_url = f'https://open.er-api.com/v6/latest/USD?apikey={api_key}'
response = requests.get(api_url)
data = response.json()
exchange_rates = data['rates']

# Define a mapping between currency symbols in your data and API symbols
currency_symbol_mapping = {
    '£': 'GBP',
    '€': 'EUR',
    '$': 'USD',
    '¥': 'JPY',
    'sek':'SEK', #swedish corona
    'euro':'EUR',
    'hkn' : 'HNK',
    'euros': 'EUR',
    'jpy': 'JPY',
    'hkd':'HDK',
    'isk':'ISK',
    'hkd$': 'HKD',
    'gbp£': 'GBP'
}

pattern = r'(?P<symbol_before>[£$€¥]|euros|eur|jpy|sek|euro|hkn|hkd|isk|hkd$|gbp£)?\s*(?P<value>\d{1,3}(?:,\d{3})*)\s*(?P<symbol_after>[£$€¥]|euros|jpy|sek|euro|hkn|hkd|isk|hkd$|gbp£)?'


In [11]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm
from functools import reduce
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
from itertools import product
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import *


def extract_masters(this_url):
    result_url = requests.get(this_url)
    result_soup = BeautifulSoup(result_url.text, 'html.parser')
    result_links = result_soup.find_all('a', {'class': 'courseLink'})
    result_list = []
    for item in result_links:
        result_list.append(item['href'])
    return result_list



def parse_html(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)
    parsed_dfs = []
    # Iterate through all folders and subfolders using os.walk
    for folder_path, _, file_names in os.walk(folder_path):
        # Check if there are files in the current folder
        if file_names:
            # Iterate through each file in the current folder
            for file_name in file_names:
                file_path = os.path.join(folder_path, file_name)

                # Store the information only is the dictionary is not empty (has at list a name course)
                try:
                    # Parse the file and append the result to the list
                    parsed_df = custom_functions.parser(file_path)
                    parsed_dfs.append(parsed_df)
                except Exception as e:
                    # Print the file path when an exception occurs
                    print(f"Error parsing file: {file_path}")
                    # print(f"Error details: {e}")

    # Concatenate all DataFrames in the list
    concatenated_df = pd.concat(parsed_dfs, ignore_index=True)
    return concatenated_df


# Function to remove stopwords and punctuation from a text
def clean(text):
    """
    The following function returns the filtered element for each column of a dataframe.
    Filtering operation consists in removing punctuation and removing stopwords given text with lower case
    """
    words = word_tokenize(str(text))
    # Remove punctuation using NLTK and string.punctuation
    filtered_words = [word for word in words if word not in string.punctuation + "'’...?'+,-'‘“”„•…›✓"]
    # Remove stopwords
    filtered_words = [word for word in filtered_words if word.lower() not in stop_words]
    return ' '.join(filtered_words)



# Function to convert any currency to the common currency (USD in this case)
def convert_to_common_currency(target_currency='USD',currency_symbol = '£',amount=0):
    try:
        # Map the currency symbol to the API symbol
        api_currency_symbol = currency_symbol_mapping.get(currency_symbol)

        if not api_currency_symbol:
            return None

        # Extract the exchange rate from the pre-fetched rates
        exchange_rate = exchange_rates[api_currency_symbol]

        # Remove the currency symbol and commas, then convert to float
        amount = float(amount.replace(',', ''))

        # Convert to USD using the obtained exchange rate
        amount_target_currency = amount/(exchange_rate)
        return round(amount_target_currency,2)
        #return currency_symbol

    except Exception as e:
       return None


def return_cost(text):
    """
    return the maximum fees converted to USD given a text (description column)
    """
    # Return None if the input is not a string
    if not isinstance(text, str):
        return None  

    matches = re.finditer(pattern, text)
    converted_list = []

    for match in matches:
        value = match.group('value')
        if match.group('symbol_before'):
            symbol_before = match.group('symbol_before')
            # Combine symbol_before, value, and symbol_after into a single string
            converted_list.append(convert_to_common_currency(currency_symbol = symbol_before,amount=value.replace(',','')))

        elif match.group('symbol_after'):
            symbol_after = match.group('symbol_after')
            converted_list.append(convert_to_common_currency(currency_symbol= symbol_after,amount=value.replace(',','')))

    # Drop None values using a list comprehension
    if len(converted_list)>=1:
        filtered_list = [value for value in converted_list if value is not None]
        if len(filtered_list)>=1:
            return(max(filtered_list))




In [12]:
text_columns = processed_df.select_dtypes(include='object').columns
# Return the list of currency symbols and costs encountered in 'fees' field through CONVERT TO COMMON CURRENCY function
raw_fees['fees (USD)'] = raw_fees['fees'].apply(lambda x: return_cost(x) if x is not None else None)


In [13]:
processed_df['fees']= raw_fees['fees (USD)']
processed_df.rename(columns={'fees': 'fees (USD)'}, inplace=True)


### Others

In [14]:
processed_df['description']= processed_df.description.apply(lambda row: word_tokenize(row))
processed_df.description.head()

0    [3d, visualis, anim, play, role, mani, areas, ...
1    [7, million, peopl, estim, die, everi, year, d...
2    [analyt, toxicolog, msc, uniqu, studi, cours, ...
3    [comput, scienc, foundat, mani, excit, cutting...
4    [polit, uncertainti, financ, recruit, demands,...
Name: description, dtype: object

In [15]:
vocabulary = set()
processed_df.description.apply(lambda row: [vocabulary.add(word) for word in row])

0       [None, None, None, None, None, None, None, Non...
1       [None, None, None, None, None, None, None, Non...
2       [None, None, None, None, None, None, None, Non...
3       [None, None, None, None, None, None, None, Non...
4       [None, None, None, None, None, None, None, Non...
                              ...                        
5974    [None, None, None, None, None, None, None, Non...
5975    [None, None, None, None, None, None, None, Non...
5976    [None, None, None, None, None, None, None, Non...
5977    [None, None, None, None, None, None, None, Non...
5978    [None, None, None, None, None, None, None, Non...
Name: description, Length: 5979, dtype: object

In [16]:
processed_df.description.head()

0    [3d, visualis, anim, play, role, mani, areas, ...
1    [7, million, peopl, estim, die, everi, year, d...
2    [analyt, toxicolog, msc, uniqu, studi, cours, ...
3    [comput, scienc, foundat, mani, excit, cutting...
4    [polit, uncertainti, financ, recruit, demands,...
Name: description, dtype: object

In [17]:
vocabulary_list = list(vocabulary)
vocabulary_dict = {word: index for index, word in enumerate(vocabulary_list)}
vocabulary_df=pd.DataFrame(list(vocabulary_dict.items()), columns=['Word', 'Id'])
print(vocabulary_df.head())

         Word  Id
0   non-human   0
1  work-focus   1
2     -inform   2
3      occupi   3
4     running   4


In [18]:
vocabulary_reverse = vocabulary_df.copy()
print(vocabulary_reverse.head())

         Word  Id
0   non-human   0
1  work-focus   1
2     -inform   2
3      occupi   3
4     running   4


In [19]:
tqdm.pandas()
vocabulary_reverse['reverse'] = vocabulary_reverse.Word.progress_apply(lambda item: list(processed_df.loc[processed_df.description.apply(lambda row: item in row)].index))

100%|██████████| 12547/12547 [01:57<00:00, 106.44it/s]


In [20]:
vocabulary_reverse.head()

,Word,Id,reverse
0,non-human,0,[3183]
1,work-focus,1,"[3955, 4716, 4717, 4718, 4865]"
2,-inform,2,[5538]
3,occupi,3,"[744, 2448, 2921, 3803, 4290, 4960]"
4,running,4,[875]


In [21]:
vocabulary_reverse[vocabulary_reverse['reverse'].apply(lambda x: len(x) == 0)]

,Word,Id,reverse


In [22]:
inverted_index = vocabulary_reverse.set_index('Id')['reverse'].to_dict()
count = 0
for key, value in inverted_index.items():
    if count < 5:
        print(f"Key: {key}, Value: {value}")
        count += 1
    else:
        break

Key: 0, Value: [3183]
Key: 1, Value: [3955, 4716, 4717, 4718, 4865]
Key: 2, Value: [5538]
Key: 3, Value: [744, 2448, 2921, 3803, 4290, 4960]
Key: 4, Value: [875]


In [ ]:
#with open('inverted_index.json', 'w') as file:
#   json.dump(inverted_index, file)

In [23]:
import pandas as pd
import json
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import os
import re
from collections import Counter
from functools import reduce
from tqdm.notebook import tqdm
from functools import reduce
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
from itertools import product



# Function to download HTML from a URL with prefix and save it to a file
def crawler(url, output_path):
    full_url = 'https://www.findamasters.com/' + url
    try:
        response = requests.get(full_url)
        if response.status_code == 200:
            with open(output_path, 'w', encoding='utf-8') as html_file:
                html_file.write(response.text)
        else:
            print(f"Failed to download {full_url}. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {full_url}: {e}")



def parser(html_page):
    # Define your default values here
    default_values = {
    'courseName': None,
    'universityName': None,
    'facultyName': None,
    'isItFullTime': None,
    'description': None,
    'startDate': None,
    'fees': None,
    'modality': None,
    'duration': None,
    'city': None,
    'administration': None,
    'country': None,
    'url': None
}

    # Create a defaultdict with default values
    info = defaultdict(lambda: default_values)

   # Extract the text (HTML)
    with open(html_page, 'r', encoding='utf-8') as file:
        html_content = file.read()
    page_soup = BeautifulSoup(html_content, 'html.parser')

    # COURSE NAME
    page_links = page_soup.find_all('h1', {'class':'text-white course-header__course-title'})
    if page_links:
        first_h1_text = page_links[0].text
        info['courseName'] = str(first_h1_text)
    #else:
        # Handle the case when no 'h1' with 'data-permutive-title' is found
    #    print("No course name found.")
    #courseName = str(first_h1_text)

    # UNIVERSITY NAME
    # Extracting our tag of interest for the Name
    page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
    # Extracting the name of the course as string and print it
    info['universityName'] = str(page_links[0].contents[0])

    # FACULTY NAME
    page_links = page_soup.find_all('a', {'class': 'course-header__department'})
    info['facultyName'] = str(page_links[0].contents[0])

    # FULL TIME
    page_links = page_soup.find_all('a', {'class': 'inheritFont'})
    info['isItFullTime'] = str(page_links[0].contents[0])

    # SHORT DESCRIPTION
    page_links = page_soup.find('div', {'id': 'Snippet'})
    info['description'] = str(page_links.get_text(separator='\n').strip())

    # STARTING DATE
    page_links = page_soup.find('span', {'class': 'key-info__start-date'})
    info['startDate'] = str(page_links.get_text())

    # FEES
    page_links = page_soup.find('a', {'class': 'noWrap inheritFont'})
    page_links = page_soup.find('div', {'class': 'course-sections__fees'})
    if page_links:
        fees_text = page_links.get_text(separator='\n').strip()
        # Remove "Fees" from the text content
        info['fees'] = fees_text.replace('Fees', '').strip()


    # MODALITY
    page_links = page_soup.find('span', {'class': 'key-info__content key-info__qualification py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
    # Get all elements within the span using find_all
    elementsWithinSpan = page_links.find_all('a')
    info['modality'] = ' '.join([element.text.strip() for element in elementsWithinSpan])

    # DURATION
    page_links = page_soup.find('span', {'class':'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
    info['duration']=str(page_links.text)

    # CITY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
    info['city']=str(page_links.text)

    # ADMINISTRATION
    page_links = page_soup.find('span', {'class':'course-header__online-flag badge bg-white p-2 h6 shadow-sm mr-1'})
    if page_links == None:
        info['administration'] = "On Campus"
    else:
        info['administration']=str(page_links.text)

    # COUNTRY
    page_links = page_soup.find('a', {'class':'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
    info['country']=page_links.text

    #URL
    page_links = page_soup.find('link')
    info['url'] = page_links.get('href')


    return(pd.DataFrame([info]))


        
def engine(query):
    """
    Given a query made up by multiple word it returns the documents were ALL the word are found
    """
    doc_set_indexes = []
    words_in_query = query.split()

    for word in words_in_query:
        # Stemming the word
        stemmed_word = stemmer.stem(word)

        # Check if the stemmed word exists in the 'Word' column after applying stemming
        if stemmed_word in vocabulary_reverse['Word'].apply(stemmer.stem).values:
            # Get the document set indexes for the stemmed word
            indexes_for_word = vocabulary_reverse[vocabulary_reverse['Word'].apply(lambda x: x == stemmed_word)]['reverse'].values

            # Flatten the lists in 'reverse' column
            flattened_indexes = [item for sublist in indexes_for_word for item in sublist]

            # Append the flattened document set indexes to the list
            doc_set_indexes.append(flattened_indexes)
           # print(doc_set_indexes)

        else:
            print(f"Stemmed word '{stemmed_word}' not found in vocabulary_reverse")

    # Find the intersection of all document sets
    selected_doc = list(set.intersection(*map(set, doc_set_indexes)))

    # Select rows using iloc
    selected_rows = df.iloc[selected_doc]

    return selected_rows


In [24]:
df_query = df[['courseName','universityName','description','url']].copy()
df_query.head()

,courseName,universityName,description,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,3D visualisation and animation play a role in ...,https://www.findamasters.com/masters-degrees/c...
1,Air Quality Solutions - MSc,University of Leeds,Up to 7 million people are estimated to die ev...,https://www.findamasters.com/masters-degrees/c...
2,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
3,Applied Computer Science and Artificial Inte...,University of Bradford,Computer science is the foundation of many exc...,https://www.findamasters.com/masters-degrees/c...
4,Applied Economics (Banking and Financial Mar...,University of Bath,From political uncertainty to finance and recr...,https://www.findamasters.com/masters-degrees/c...


In [25]:
engine('advance knowledge')

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,administration,country,url
2,Analytical Toxicology MSc,King’s College London,Faculty of Life Sciences & Medicine,Full time,The Analytical Toxicology MSc is a unique stud...,See Course,Please see the university website for further ...,MSc,Full-time: One year,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
7,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,"UK: £18,000 (Total) \r\nInternational: £34,750...",MSc,1 year full time,Leeds,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2056,Engineering with Management - MSc,University of Bristol,Faculty of Engineering,Full time,The MSc in Engineering with Management is desi...,September,Please see the university website for further ...,MSc,1 year full-time,Bristol,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
10,Addictions MSc,King’s College London,"Institute of Psychiatry, Psychology and Neuros...",Full time,Join us for an online session for prospective ...,September,Please see the university website for further ...,MSc,One year FT,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2069,Enhanced Radiotherapy and Oncology Practice ...,Sheffield Hallam University,Postgraduate Courses,Part time,Enhance your knowledge of specific radiotherap...,"September, January",For part-time study the fee will be calculated...,MSc,3 years part time distance learning,Sheffield,Online,United Kingdom,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,Engineering Management (MSc),University of Gloucestershire,School of Computing and Engineering,Full time,This course will equip you with the necessary ...,September,Please see the university website for further ...,MSc,See website for details,Gloucester,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2032,Engineering Management MSc,University of Bradford,Faculty of Engineering & Digital Technologies,Full time,The MSc Engineering Management programme will ...,"September, January",Please see the university website for further ...,MSc,1 year Full time,Bradford,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
2034,Engineering Management MSc,Middlesex University,Faculty of Science and Technology,Full time,The career of an engineering project manager i...,October,"UK students \r\nFull-time students: £10,500 \r...",MSc,1 year full-time; 2 years part-time,London,On Campus,United Kingdom,https://www.findamasters.com/masters-degrees/c...
4083,M2 Research in Management and Innovation,Université Côte d’Azur,UCA International MSc Program,Full time,"Our modern industry structures, value chains a...",September,The tuition currently stands at €243 as well a...,MSc,2 years,Nice,On Campus,France,https://www.findamasters.com/masters-degrees/c...


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(processed_df['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=processed_df.index, columns=tfidf.get_feature_names_out())
print(tfidf_data.head())

c:\Users\Matt\anaconda3\envs\Py_11\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


     '  'all  'appli  'bench  'big  'chip  'credits   'd  'disability  'educ  \
0  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
1  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
2  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
3  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   
4  0.0   0.0     0.0     0.0   0.0    0.0       0.0  0.0          0.0    0.0   

   ...  ﻿over  ﻿prove  ﻿regist  ﻿th  ﻿the  ﻿theoret  ﻿thi  ﻿want   ﻿y   ﻿﻿  
0  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
1  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
2  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
3  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  
4  ...    0.0     0.0      0.0  0.0   0.0       0.0   0.0    0.0  0.0  0.0  

[5 rows x 12547 columns]


In [28]:
inverted_index_2 = {}
for word in tfidf_data.columns:
    word_docs = tfidf_data.index[tfidf_data[word] > 0].tolist()
    word_doc_scores = [(doc, tfidf_data.loc[doc, word]) for doc in word_docs]
    inverted_index_2[word] = word_doc_scores
count = 0
for word, doc_scores in inverted_index_2.items():
    print(f"Word: {word}")
    print("Document Scores:")
    print(doc_scores)
    count += 1
    if count == 10:
        break

Word: '
Document Scores:
[(3244, 0.2724922372887123)]
Word: 'all
Document Scores:
[(3871, 0.18492587961352233)]
Word: 'appli
Document Scores:
[(2325, 0.1722113554646612)]
Word: 'bench
Document Scores:
[(333, 0.3466489203597607)]
Word: 'big
Document Scores:
[(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)]
Word: 'chip
Document Scores:
[(1902, 0.1992739368962018)]
Word: 'credits
Document Scores:
[(2317, 0.18303193893688977), (5926, 0.08382342995897131)]
Word: 'd
Document Scores:
[(2160, 0.194367236072988), (4863, 0.1419174496168122)]
Word: 'disability
Document Scores:
[(4783, 0.17222181095019545)]
Word: 'educ
Document Scores:
[(2618, 0.2647135148766833)]


In [29]:
new_inverted_index = {}
for word, doc_scores in inverted_index_2.items():
    word_id = vocabulary_df[vocabulary_df['Word'] == word]['Id'].values
    if len(word_id) > 0:
        word_id = word_id[0]
        new_inverted_index[word_id] = doc_scores

In [30]:
n = 10
partial_inverted_index = {k: new_inverted_index[k] for k in list(new_inverted_index.keys())[:n]}
print(partial_inverted_index)

{1156: [(3244, 0.2724922372887123)], 985: [(3871, 0.18492587961352233)], 12265: [(2325, 0.1722113554646612)], 12276: [(333, 0.3466489203597607)], 6488: [(1217, 0.2404462962515215), (1225, 0.1886881267487287), (1290, 0.2107926430819876), (5692, 0.17695012088105525)], 6626: [(1902, 0.1992739368962018)], 303: [(2317, 0.18303193893688977), (5926, 0.08382342995897131)], 78: [(2160, 0.194367236072988), (4863, 0.1419174496168122)], 7438: [(4783, 0.17222181095019545)], 2191: [(2618, 0.2647135148766833)]}


### Cosinesim

In [31]:
df_query_processed= processed_df[['courseName','universityName','description','url']].copy()

In [32]:
df_query_processed.head()

,courseName,universityName,description,url
0,3d design virtual environments msc,glasgow caledonian university,"[3d, visualis, anim, play, role, mani, areas, ...",https //www.findamasters.com/masters-degrees/c...
1,air quality solutions msc,university leeds,"[7, million, peopl, estim, die, everi, year, d...",https //www.findamasters.com/masters-degrees/c...
2,analytical toxicology msc,king college london,"[analyt, toxicolog, msc, uniqu, studi, cours, ...",https //www.findamasters.com/masters-degrees/c...
3,applied computer science artificial intelligen...,university bradford,"[comput, scienc, foundat, mani, excit, cutting...",https //www.findamasters.com/masters-degrees/c...
4,applied economics banking financial markets on...,university bath,"[polit, uncertainti, financ, recruit, demands,...",https //www.findamasters.com/masters-degrees/c...


In [33]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text: text) # , max_df=0.1
results = tfidf.fit_transform(df_query_processed['description'])
result_dense = results.todense()
tfidf_data = pd.DataFrame(result_dense.tolist(), index=df_query_processed.index, columns=tfidf.get_feature_names_out())

c:\Users\Matt\anaconda3\envs\Py_11\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [34]:
def rank_documents(query1):
    """
    Given a query, computes the tfidf for that query and evaluate the cosine similarity con for query given
    the document extracting with engine function. Returns the top-k documents
    """
    ## COMPUTING THE IFIDF FOR THE QUERY

   # Tokenize the query and save the stammed words for computinf tfidf
    tokens = word_tokenize(query1)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    tfidf_vectorizer = TfidfVectorizer()

    # Combine the stemmed tokens considering it single string (document)
    stemmed_query = ' '.join(stemmed_tokens)

    # Fit the vectorizer on the list of queries and transform the query into a tfidf vector
    query_tfidf = tfidf_vectorizer.fit_transform([stemmed_query])

    # Convert the TF-IDF matrix to a dense pandas DataFrame
    tfidf_query = pd.DataFrame(query_tfidf.todense(), index=['query'], columns=tfidf_vectorizer.get_feature_names_out())

    ## OBTAINE THE TFIDF BEWTEEN THE QUERY AND THE DOCUMENT
    # Initialize a heap for the k-top results
    k_top_results = []

    # Iterate over the documents
    for idx, document_text in enumerate(engine(query1)['description']):
        # Tokenize and preprocess the document text
        document_tokens = [stemmer.stem(word) for word in word_tokenize(document_text)]
        document_text_processed = ' '.join(document_tokens)

        # Transform the document text using the fitted vectorizer
        tfidf_document = tfidf_vectorizer.transform([document_text_processed])

        # Compute cosine similarity between the query and the document
        similarity_score = cosine_similarity(tfidf_query, tfidf_document)[0, 0]

       ## USING A HEAP TO HAVE THE TOP-K DOCUMENTS
        # Append the similarity score and document index to the k_top_results list
        k_top_results.append((round(similarity_score,5), engine(query1)['description'].index[idx]))


    # Retrieve the k-top results from the heap using nlargest
    k_top_results = heapq.nlargest(10, k_top_results, key=lambda x: x[0])
    rank_df = pd.DataFrame()

    for cossim, indexsorted in k_top_results:
        # Return the information about the document sorted by cosine similarity
            document_info = df_query.loc[[indexsorted]].copy()
#            return document_info
            document_info['Similarity'] = cossim
            rank_df = pd.concat([rank_df, document_info])

    # Eventually we could reset the index of the final DataFrame
    # rank_df = rank_df.reset_index(drop=True)

    return rank_df

In [35]:
rank_documents("artificial intelligence data analytics")

,courseName,universityName,description,url,Similarity
5783,Artificial Intelligence with Data Analytics MSc,Teesside University,The MSc Artificial Intelligence with Data Anal...,https://www.findamasters.com/masters-degrees/c...,1.00000
4722,Master of Science in Artificial Intelligence,Asia Pacific University of Technology & Innova...,This programme is specifically designed to pro...,https://www.findamasters.com/masters-degrees/c...,0.98995
4723,Master of Science in Artificial Intelligence...,Asia Pacific University of Technology & Innova...,This programme is specifically designed to pro...,https://www.findamasters.com/masters-degrees/c...,0.98995
4725,Master of Science in Big Data Analytics and ...,Atlantic Technological University,Our Master of Science in Computing in Big Data...,https://www.findamasters.com/masters-degrees/c...,0.98198
5756,Artificial Intelligence and Data Analytics,Loughborough University London,The Artificial Intelligence and Data Analytics...,https://www.findamasters.com/masters-degrees/c...,0.94868
1282,Data Science (with specialisation in Artific...,Newcastle University,"MSc - full time: 12 months, part time: 24 mont...",https://www.findamasters.com/masters-degrees/c...,0.93831
1291,Data Science and Analytics MSc,University of Westminster,This course (previously Business Intelligence ...,https://www.findamasters.com/masters-degrees/c...,0.89443
5776,Artificial Intelligence MSc,Teesside University,Artificial intelligence (AI) is increasingly i...,https://www.findamasters.com/masters-degrees/c...,0.86603
4760,Master of Science in Computing and Data Anal...,"Saint Mary’s University, Canada",Gartner projects that over 75% of enterprises ...,https://www.findamasters.com/masters-degrees/c...,0.86603
3323,"Business Analytics (part-time, online) - MSc",Imperial College London,Gain in-demand data analytics skills without p...,https://www.findamasters.com/masters-degrees/c...,0.85749


In [36]:
ranked_universities = rank_documents("artificial intelligence data analytics")


In [37]:
ranked_universities

,courseName,universityName,description,url,Similarity
5783,Artificial Intelligence with Data Analytics MSc,Teesside University,The MSc Artificial Intelligence with Data Anal...,https://www.findamasters.com/masters-degrees/c...,1.00000
4722,Master of Science in Artificial Intelligence,Asia Pacific University of Technology & Innova...,This programme is specifically designed to pro...,https://www.findamasters.com/masters-degrees/c...,0.98995
4723,Master of Science in Artificial Intelligence...,Asia Pacific University of Technology & Innova...,This programme is specifically designed to pro...,https://www.findamasters.com/masters-degrees/c...,0.98995
4725,Master of Science in Big Data Analytics and ...,Atlantic Technological University,Our Master of Science in Computing in Big Data...,https://www.findamasters.com/masters-degrees/c...,0.98198
5756,Artificial Intelligence and Data Analytics,Loughborough University London,The Artificial Intelligence and Data Analytics...,https://www.findamasters.com/masters-degrees/c...,0.94868
1282,Data Science (with specialisation in Artific...,Newcastle University,"MSc - full time: 12 months, part time: 24 mont...",https://www.findamasters.com/masters-degrees/c...,0.93831
1291,Data Science and Analytics MSc,University of Westminster,This course (previously Business Intelligence ...,https://www.findamasters.com/masters-degrees/c...,0.89443
5776,Artificial Intelligence MSc,Teesside University,Artificial intelligence (AI) is increasingly i...,https://www.findamasters.com/masters-degrees/c...,0.86603
4760,Master of Science in Computing and Data Anal...,"Saint Mary’s University, Canada",Gartner projects that over 75% of enterprises ...,https://www.findamasters.com/masters-degrees/c...,0.86603
3323,"Business Analytics (part-time, online) - MSc",Imperial College London,Gain in-demand data analytics skills without p...,https://www.findamasters.com/masters-degrees/c...,0.85749


### EX 3

In [62]:
import numpy as np
from geopy.geocoders import Nominatim
import folium 
from geopy import distance
from folium.plugins import HeatMap
import branca
import branca.colormap as cm


geolocator = Nominatim(user_agent="my_user_agent")


### Define a new metric for scoring the universities

To make the score more personal, the user (student) will be asked to input 2 new info:
- "In which city would you prefer to study?" It is possible to input every city in the world, and even other universities. However, the main idea is that the score will be lowered should the university be far from where the student wants to study. 
- "What is the maximum distance from the university to the city centre (in Kilometres) that you are willing to accept?" This will set a MaxDistance from the input above. Let's say for instance that i want to study in London and be 20 Km far from the city. The metric will take into consideration that. Every university further from the specified distance will have a negative score.

We will still use the cosine similarity as our baseline, since it gives us a starting point to evaluate how similar with the query the documents are.
Secondly, the cossim will be multiplied by 2 new parametres, which will refine the metric.

The first parametre is $$(\epsilon + 1 - (\frac{UniDistance}{MaxDistance}))$$
- Unidistance is the distance from the city, entered by the user
- Max distance is the second value entered by the user
The higher the parametre, the better the result. Note that the UniDistance can be higher than the MaxDistance, so you could expect a negative score, which significantly lowers the position of the university. $\epsilon$ is a parametre added to keep the total from being zero (for example when UniDistance = MaxDistance, the result of the formula is just $\epsilon$)

The second parametre is $$(\epsilon + 1 - (\frac{UniFees}{MaxFees}))$$
- UniFees is the cost of the university, and we will use the processed fees column from the point above.
- MaxFees is the max cost among every column
$\epsilon$ is a parametre added to keep the total from being zero (for example when UniFees = MaxFees, the result of the formula is just $\epsilon$)

So the new scoring function is defined as follows: $$SimilarityScore = CosineSimilarity(Query, Document)*(\epsilon + 1 - (\frac{UniDistance}{MaxDistance}))*(\epsilon + 1 - (\frac{UniFees}{MaxFees}))$$



### IMPORTANT INFO ON PROCESSED FEES: 

The number of universities with clean fees is way lower than the total. Just to give a rough estimation $\approx1300$ out of $5979$ universities are visualized correctly, whereas the others have a "Please navigate to another page to collect the fees". To avoid a scoring problem, as well as a visualization one, the NA values are replaced with the an interpolated value. To be precise, citing Pandas documentation "Fill NaN values using an interpolation method". We choose the "linear" method, which in this case seems to preserve the distribution of fees. 

It has to be kept in mind that the NOT every score is precisely the best for the student (because of the intepolated values), and giving an "outlier" fee to a specific university just because the parsing mechanism required a second step wasn't technically a correct choice.



In [57]:
raw_fees["fees (USD)"] = raw_fees["fees (USD)"].interpolate(method = "linear")#interpolate the missing values (na values with the linear method)
raw_fees["fees (USD)"] = round(raw_fees["fees (USD)"].fillna(raw_fees["fees (USD)"].mean()),2) #fill the remaining na values with the mean, since a value was missing
max_fees = raw_fees["fees (USD)"].max() #compute the max fees

In [66]:
def rank_documents_advanced(query1,max_fees):
    """
    Given a query, computes the tfidf for that query and evaluate the cosine similarity con for query given
    the document extracting with engine function. Returns the top-k documents
    """

    central_point = input("In which city would you prefer to study?")
    max_distance = int(input("What is the maximum distance from the university to the city centre (in Kilometres) that you are willing to accept?"))
    student_point = geolocator.geocode(central_point) #get coordinates 
    student_coordinates = [student_point.latitude, student_point.longitude] #add the coordinates to a list

    #define the epsilon value for the new metric, so that the score is never zero
    epsilon = 0.05


    ## COMPUTING THE IFIDF FOR THE QUERY
    # Tokenize the query and save the stammed words for computinf tfidf
    tokens = word_tokenize(query1)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    tfidf_vectorizer = TfidfVectorizer()

    # Combine the stemmed tokens considering it single string (document)
    stemmed_query = ' '.join(stemmed_tokens)

    # Fit the vectorizer on the list of queries and transform the query into a tfidf vector
    query_tfidf = tfidf_vectorizer.fit_transform([stemmed_query])

    # Convert the TF-IDF matrix to a dense pandas DataFrame
    tfidf_query = pd.DataFrame(query_tfidf.todense(), index=['query'], columns=tfidf_vectorizer.get_feature_names_out())

    ## OBTAINE THE TFIDF BEWTEEN THE QUERY AND THE DOCUMENT
    # Initialize a heap for the k-top results
    k_top_results = []

    # Iterate over the documents
    for idx, document_text in enumerate(engine(query1)['description']):

        # Tokenize and preprocess the document text
        document_tokens = [stemmer.stem(word) for word in word_tokenize(document_text)]
        document_text_processed = ' '.join(document_tokens)

        # Transform the document text using the fitted vectorizer
        tfidf_document = tfidf_vectorizer.transform([document_text_processed])

        
        
        uni_name_info = df_query.loc[[idx]].copy()  
        univ = uni_name_info["universityName"][idx] #get the university name
        uni_fees = raw_fees['fees (USD)'][idx] #get the fees

    
        uni_point = geolocator.geocode(univ) #get coordinates 
        uni_coordinates = [uni_point.latitude, uni_point.longitude] #add the coordinates to a list
        uni_distance = distance.distance(uni_coordinates,student_coordinates).km #compute the distance between the university and the student's specified location
        
        print("The distance from your specified location is",round(uni_distance,2),"Km")

        
        #Compute the new score between the query and the document
        #print(uni_distance/max_distance)
        print("Uni_fees / Max_fees =", uni_fees/max_fees)
        print("Uni_fees =", uni_fees)
        similarity_score = (cosine_similarity(tfidf_query, tfidf_document)[0, 0])*(epsilon+1-(uni_distance/max_distance))*(epsilon+1-(uni_fees/max_fees))
        
        #Since the score has to be somewhat personal, we let the student decide the maximum distance from the university. if the distance is greater than the specified distance
        #the score will be negative. A negative score will not be one of the top choices.
        #The negative score is due to the fact that the second parenthesis is negative, since the distance can be greater than the maximum distance, which is an input from the user

        print("Newmetric =",(epsilon + 1-(uni_distance/max_distance))*(epsilon + 1-(uni_fees/max_fees)))
        print(similarity_score)
       

       ## USING A HEAP TO HAVE THE TOP-K DOCUMENTS
        # Append the similarity score and document index to the k_top_results list
        k_top_results.append((round(similarity_score,5), engine(query1)['description'].index[idx]))


    # Retrieve the k-top results from the heap using nlargest
    k_top_results = heapq.nlargest(10, k_top_results, key=lambda x: x[0])
    rank_df = pd.DataFrame()

    for cossim, indexsorted in k_top_results:
        # Return the information about the document sorted by cosine similarity
            document_info = df_query.loc[[indexsorted]].copy()
            print(document_info.columns)
#            return document_info
            document_info['Similarity'] = cossim
            rank_df = pd.concat([rank_df, document_info])

    # Eventually we could reset the index of the final DataFrame
    # rank_df = rank_df.reset_index(drop=True)

    return rank_df

In [67]:
ranked_universities = rank_documents_advanced("advanced knowledge artificial intelligence",max_fees)

The distance from your specified location is 556.24 Km
Uni_fees / Max_fees = 0.2674267477949815
Uni_fees = 15251.08
Newmetric = -3.5312972090178447
-3.1239682095715184
The distance from your specified location is 273.5 Km
Uni_fees / Max_fees = 0.6319942485402164
Uni_fees = 36042.0
Newmetric = -0.7043601787229066
-0.683740276363959
The distance from your specified location is 1.1 Km
Uni_fees / Max_fees = 0.4266250504129478
Uni_fees = 24330.0
Newmetric = 0.6477148213842371
0.6120329778118974
The distance from your specified location is 277.27 Km
Uni_fees / Max_fees = 0.2212558522856792
Uni_fees = 12618.0
Newmetric = -1.4277144987417554
-1.4277144987417554
The distance from your specified location is 153.35 Km
Uni_fees / Max_fees = 0.01586911922004594
Uni_fees = 905.0
Newmetric = -0.49998611825208383
-0.4724424742426952
The distance from your specified location is 557.96 Km
Uni_fees / Max_fees = 0.26521594276596117
Uni_fees = 15125.0
Newmetric = -3.554789319787068
-3.3514875116146
The dis

Add to the ranked_universities dataframe fees,cities and countries, for the next question

In [69]:

ranked_universities["fees"] = raw_fees['fees (USD)']  #add fees to the dataframe
ranked_universities["country"] = df["country"] #add country to the dataframe
ranked_universities["city"] = df["city"] #add city to the dataframe


In [70]:
ranked_universities

,courseName,universityName,description,url,Similarity,fees,country,city
5774,Artificial Intelligence MSc,University of East London,This programme is designed for those who want ...,https://www.findamasters.com/masters-degrees/c...,0.61203,9911.0,United Kingdom,London
5140,Advanced Computer Science (Artificial Intell...,University of Leeds,From software agents used in networking system...,https://www.findamasters.com/masters-degrees/c...,-0.47244,39803.0,United Kingdom,Leeds
4589,Master of Management in Artificial Intelligence,York University (Canada),"Looking for Coders, Innovators and AI Visionar...",https://www.findamasters.com/masters-degrees/c...,-0.68374,55.0,Canada,Toronto
5775,Artificial Intelligence MSc,University of Huddersfield,"In the UK, we have seen a huge expansion in ar...",https://www.findamasters.com/masters-degrees/c...,-1.42771,6958.0,United Kingdom,Huddersfield
1913,Electronics Engineering,Linköping University,This programme focuses on the design of integr...,https://www.findamasters.com/masters-degrees/c...,-1.96681,26741.0,Sweden,Linkoping
1356,Data Science with Artificial Intelligence MSc,University of Exeter,Overview\r\nDesigned for those interested in l...,https://www.findamasters.com/masters-degrees/c...,-3.12397,19098.0,United Kingdom,Exeter
5749,Artificial Intelligence (MSc),Queen’s University Belfast,In the last decade the advances in Artificial ...,https://www.findamasters.com/masters-degrees/c...,-3.35149,617.0,United Kingdom,Belfast


### Performance review

The new metric works in a different way from the original, and the output can be interpreted in the following way:
- If different universities are in the same radius specified by the user, the ranking will be more precise than the original one, since the higher overall score will be destined for the universities with the highest value in each parenthesis.
- If the university is far from the specified location, its score will be lower, so is is indeed a very important metric to consider.

### EX 4

### For this visualization task we used the Folium library, with the geolocator Nominatim.

- The first one gives us the OpenStreetMap with marked coordinates, colors and information about the Master courses.
- The second one let us compute the coordinates of every university, and the geodesic distance between two points (which are the city where the student wants to study and the university)

Following the same line of argument as point Q3, the student will be asked to input the city where he is most likely to study (and this will centre the map on the specified city, with a certain zoom). The map will then show the universities, sorted by color and with a legend on the top right corner:
- The greener the "Circle Marker" the less fees the student has to pay (Also take into consideration how some values of the fees have been processed in point Q3)

Note that the map is interactive, you can zoom in and out, and will show the following information if you click on a marker:
- University
- Country
- City
- Course Name
- Fees
- Distance from the user input
- The new personalized score, so that the student can evaluate every option in the map

In [ ]:
ranked_universities = ranked_universities.sort_values(by=['fees'], ascending=True) #sort ranked_universities by fees, from lowest to highest, so that the map will show the cheapest universities first.

In [82]:
coordinates = []
university_list = []

for element in ranked_universities.universityName:
    univ = element
    university_list.append(univ)
    loc = geolocator.geocode(univ) #we need to offset the coordinates by a random factor so that if two Msc are from the same university they are shown in the map
    offset_univ_coordinates = (loc.latitude + np.random.uniform(10**(-3), 10**(-3.5)),loc.longitude + np.random.uniform(10**(-3), 10**(-3.5))) #offest the coordinates by a random factor
    coordinates.append(offset_univ_coordinates) #add the coordinates to a list

print(coordinates)
print(university_list)

[(43.774573332496544, -79.49937024181418), (54.58510007000635, -5.933015144153491), (53.64339754744312, -1.7776054841026896), (51.54524367123879, 0.010154271951044604), (50.73766642218661, -3.5342402343230725), (58.39855675134673, 15.575847536008956), (53.807716113793845, -1.5558903882838229)]
['York University (Canada)', 'Queen’s University Belfast', 'University of Huddersfield', 'University of East London', 'University of Exeter', 'Linköping University', 'University of Leeds']


In [167]:
ranked_universities = ranked_universities.reset_index() #reindex ranked_universities, so that Foliun works properly

In [172]:

initial_test = input() #ask the user to input the city where he/she would like to study

initial_coordinates = geolocator.geocode(initial_test) #get coordinates
initial_point = [initial_coordinates.latitude, initial_coordinates.longitude] #add the coordinates to a list


uni_map = folium.Map(location=initial_point, zoom_start=12) #specify the initial point (city) and the zoom level
colormap = cm.LinearColormap(colors=["darkgreen",'green','yellow','orange',"red","darkred"],index=ranked_universities.index,vmin=(min(ranked_universities.fees)),vmax=max(ranked_universities.fees)) #indexing the colormap with the index of the dataframe
colormap_show = cm.LinearColormap(colors=["darkgreen",'green','yellow','orange',"red","darkred"],vmin=(min(ranked_universities.fees)),vmax=max(ranked_universities.fees)) #remove index to show the correct colormap
folium.Circle(initial_point,radius = 5000).add_to(uni_map) #add a circle to the map, with a radius of 5km (default value, but the score funcition will take into consideration the user input), to show the maximum distance from the city that the user is willing to accept

for coordinate, idx in zip(coordinates,ranked_universities.index):
    html = '''
    <b>University: </b>{university}<br>
    <b>Country: </b>{country}<br>
    <b>City: </b>{city}<br>
    <b>Course Name:</b>{coursename}<br>
    <b>Fees: </b>{fees} $<br>
    <b>Distance from your specified location: </b>{distance} Km <br>
    <b>Score: </b>{newscore}
    
    '''.format(university=ranked_universities.universityName[idx],index=ranked_universities.fees,country=ranked_universities.country[idx],city=ranked_universities.city[idx],coursename=ranked_universities.courseName[idx],fees=ranked_universities.fees[idx],distance=round(distance.distance(coordinate,initial_point).km,2),newscore = ranked_universities.Similarity[idx])
    
    new_popup = folium.Popup(html, parse_html=False, max_width=1000) #create a popup with the function Popup, which will show the information about the university when clicked on the marker

    folium.CircleMarker(
        location=coordinate,
        fill = True,
        popup=new_popup,
        color = colormap(idx),
        fill_color = colormap(idx)           
    ).add_to(uni_map)

    

uni_map.add_child(colormap_show)
    
uni_map